<a href="https://colab.research.google.com/github/neroblack4life/tradingview-yahoo-finance/blob/main/AnchoredVWAP_Pytvlwcharts_Flask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet --upgrade --no-cache-dir git+https://github.com/TechfaneTechnologies/pytvlwcharts.git git+https://github.com/StreamAlpha/tvdatafeed.git pandas_ta flask-cloudflared

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 KB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 KB 107.3 MB/s eta 0:00:00


In [ ]:
import sys
import pytz
import requests
import warnings
import numpy as np
import pandas as pd
import datetime as dt
import pandas_ta as ta
from dateutil import tz
from typing import Union
from copy import deepcopy
from pytvlwcharts import *
from tvDatafeed import TvDatafeed, Interval
from flask import (
    Flask,
    jsonify,
    request,
    Response,
    render_template_string,
)
from flask_cloudflared import run_with_cloudflared
warnings.filterwarnings("ignore", "warn", category=DeprecationWarning)

In [ ]:
from_zone = tz.gettz("UTC")
to_zone = tz.gettz("Asia/Kolkata")
_headers = {
    "User-Agent": requests.get(
        "https://techfanetechnologies.github.io/latest-user-agent/user_agents.json"
    ).json()[-2]
}

In [ ]:
def getChart(notebook_mode:bool=True) -> Chart:
    return Chart(
        notebook_mode=notebook_mode,
        layout=LayoutOptions(
            background=SolidColor(color="#222"),
            background_color="#222",
            text_color="#DDD",
        ),
        grid=GridOptions(
            horz_lines=GridLineOptions(
                color="#444",
                style=LineStyle.LINESTYLE_0,
                visible=False,
            ),
            vert_lines=GridLineOptions(
                color="#444",
                style=LineStyle.LINESTYLE_0,
                visible=False,
            ),
        ),
        width=1200,
        height=500,
        time_scale=TimeScaleOptions(
            seconds_visible=True,
            time_visible=True,
            border_visible=True,
            border_color="#71649C",
        ),
        price_scale=PriceScaleOptions(
            auto_scale=True,
            border_visible=True,
            border_color="#71649C",
            scale_margins=PriceScaleMargins(bottom=0.4, top=0.1),
        ),
        overlay_price_scales=OverlayPriceScaleOptions(
            scale_margins=PriceScaleMargins(bottom=0, top=0.7)
        ),
    )

In [ ]:
def AnchoredVwap(
    df: pd.DataFrame, anchor: pd.Timestamp, anchor_timeframe: str = "D"
) -> pd.DataFrame:
    df["time"] = df.index.tz_localize(pytz.utc).tz_convert(
        pytz.timezone("Asia/Kolkata")
    )
    df.set_index(pd.DatetimeIndex(df["time"]), inplace=True)
    df.drop(columns=["time", "symbol"], inplace=True)
    df = pd.concat([df, df.ta.vwap(anchor=anchor_timeframe)], axis=1)
    tpp = ((df["high"] + df["low"] + df["close"]) * df["volume"]) / 3
    df["anchored_VWAP"] = (
        tpp.where(df.index >= anchor).groupby(df.index >= anchor).cumsum()
        / df["volume"].where(df.index >= anchor).groupby(df.index >= anchor).cumsum()
    )
    df.reset_index(inplace=True)
    df["time"] = (df.time.view(np.int64) // 1000000000) + 19800
    return df

In [ ]:
def getOHLCVData(
    symbol: str = "NIFTY",
    exchange: str = "NSE",
    chart_interval: Interval = Interval.in_15_minute,
    n_bars: int = 5000,
    anchor: pd.Timestamp = pd.to_datetime("2023-02-10 09:45:00+0530"),
    anchor_timeframe: str = "15min",
    is_future: bool = True,
) -> pd.DataFrame:
    tv = TvDatafeed()
    data = tv.get_hist(
        symbol=symbol.upper(),
        exchange=exchange.upper(),
        interval=chart_interval,
        n_bars=n_bars,
        fut_contract=int(is_future),
    )
    df = AnchoredVwap(
        df=data,
        anchor=anchor,
        anchor_timeframe=anchor_timeframe,
    )
    df["volColor"] = np.select(
        [
            (df["open"] < df["close"]),
            (df["open"] > df["close"])
        ],
        ["#26a69a", "#ef5350"]
    )
    return df

In [ ]:
def PrepareChart(
    symbol: str = "NIFTY",
    exchange: str = "NSE",
    chart_interval: Interval = Interval.in_15_minute,
    n_bars: int = 5000,
    anchor: pd.Timestamp = pd.to_datetime("2023-02-10 09:45:00+0530"),
    anchor_timeframe: str = "15min",
    is_future: bool = True,
    notebook_mode: bool = True,
) -> Union[str, Chart]:
    chart = getChart(notebook_mode)
    df = getOHLCVData(
        symbol=symbol,
        exchange=exchange,
        chart_interval=chart_interval,
        n_bars=n_bars,
        anchor=anchor,
        anchor_timeframe=anchor_timeframe,
        is_future=is_future,
    )
    chart.mark_candlestick(
        upColor="#26a69a",
        downColor="#ef5350",
        borderVisible=False,
        wickUpColor="#26a69a",
        wickDownColor="#ef5350",
        data=df[["time", "open", "high", "low", "close"]],
        title=symbol + "1!" if is_future else symbol,
        series_name="ohlc",
    )
    chart.mark_histogram(
        data=df[["time", "volume", "volColor"]].rename(
            columns={"volume": "value", "volColor": "color"}
        ),
        title="Volume",
        priceFormat={"type": "volume"},
        priceScaleId="",
        series_name="volume",
    )
    chart.mark_line(
        data=df.dropna()[["time", "anchored_VWAP"]].rename(
            columns={"anchored_VWAP": "value"}
        ),
        title="AnchoredVWAP",
        color="orange",
        lineWidth=LineWidth.LINEWIDTH_3,
        series_name="avwap",
    )
    if notebook_mode:
        return chart
    else:
        return (
            chart._repr_html_()
            .replace(": 500", ": document.body.offsetHeight")
            .replace(": 1200", ": document.body.offsetWidth")
        )

In [ ]:
app = Flask(__name__)

# Auto Background Fetching and updation of data from this endpoint.
# http://127.0.0.1:5000/data/NIFTY
# https://rachel-exclusive-context-occupied.trycloudflare.com/data/NIFTY
@app.route("/data/<symbol>", methods=["GET"])
def data(symbol):
    df = getOHLCVData(
        symbol=symbol,
        exchange="NSE",
        chart_interval=Interval.in_1_minute,
        anchor=pd.to_datetime("2023-02-17 09:15:00+0530"),
        anchor_timeframe="1min",
        n_bars=5000,
        is_future=True,
    )
    return jsonify(
        {
            "ohlc": df[["time", "open", "high", "low", "close"]]
            .tail(2)
            .to_dict(orient="records")[:1],
            "volume": df[["time", "volume", "volColor"]]
            .rename(columns={"volume": "value", "volColor": "color"})
            .tail(2)
            .to_dict(orient="records")[:1],
            "avwap": df.dropna()[["time", "anchored_VWAP"]]
            .rename(columns={"anchored_VWAP": "value"})
            .tail(2)
            .to_dict(orient="records")[:1],
        }
    )

# Main Chart Dispaly Endpoint
# http://127.0.0.1:5000/chart/NIFTY
# https://rachel-exclusive-context-occupied.trycloudflare.com/chart/NIFTY
@app.route("/chart/<symbol>", methods=["GET"])
def chart(symbol):
    return render_template_string(
        PrepareChart(
            symbol=symbol,
            exchange="NSE",
            chart_interval=Interval.in_1_minute,
            anchor=pd.to_datetime("2023-02-17 09:15:00+0530"),
            anchor_timeframe="1min",
            n_bars=5000,
            is_future=True,
            notebook_mode=False,
        )
    )

run_with_cloudflared(app)  # Open a Cloudflare Tunnel when app is run

# if __name__ == "__main__":
app.run()

# Visit the Running on Url which have `.trycloudflare.com` with suffix /chart/SYMBOLNAME
# For Example https://fiber-theory-releases-boston.trycloudflare.com/chart/NIFTY
# Then Stay on the same page, and after 60 second, the last minute candle will be appended to the chart automatically.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on https://fiber-theory-releases-boston.trycloudflare.com
 * Traffic stats available on http://127.0.0.1:8837/metrics


/usr/local/lib/python3.8/dist-packages/pandas/core/arrays/datetimes.py:1143: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
INFO:werkzeug:127.0.0.1 - - [21/Feb/2023 06:49:41] "GET /chart/NIFTY HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Feb/2023 06:49:49] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [21/Feb/2023 06:49:50] "GET /data/NIFTY HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Feb/2023 06:49:54] "GET /chart/NIFTY HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Feb/2023 06:49:55] "GET /chart/NIFTY HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Feb/2023 06:50:02] "GET /data/NIFTY HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Feb/2023 06:50:07] "GET /chart/NIFTY HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Feb/2023 06:50:08] "GET /chart/NIFTY HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Feb/2023 06:50:13] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [21/Feb/2023 06:51:02] 